In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%%capture
!pip install rasterio

In [3]:
import rasterio
import numpy as np
from PIL import Image
from rasterio.control import GroundControlPoint
from rasterio.transform import from_gcps
from rasterio.transform import from_gcps
from osgeo import gdal
from osgeo import osr
import glob
import matplotlib.pyplot as plt
import pandas as pd
import os
import re

In [4]:
#緯度経度情報の10進数への変換
def convert_to_decimal(coord):
  deg=coord.split('°')[0]
  minute=coord.split('′')[0].split('°')[1]
  second=coord.split('′')[1].replace('″','')
  if len(second)>0:
    pass
  else:
    second=0
  coord_decimal=float(deg)+float(minute)/60+float(second)/3600

  return coord_decimal


In [5]:
#四隅に緯度経度を付与する
folder_path = "/content/drive/MyDrive/NorthKorea/"  # フォルダのパスを指定
jpg_pattern = r"res_(\w+)\.jpg"
csv_pattern = r"cood(\w+)\.csv"

jpg_files = {}
csv_files = {}

# jpgファイルを探して辞書に追加
for filename in os.listdir(folder_path):
    jpg_match = re.match(jpg_pattern, filename)
    if jpg_match:
        key = jpg_match.group(1)
        jpg_files[key] = filename
#        print(filename)

# csvファイルを探して辞書に追加
for filename in os.listdir(folder_path):
    csv_match = re.match(csv_pattern, filename)
    if csv_match:
        key = csv_match.group(1)
        csv_files[key] = filename

# 共通のキーを持つファイルを組み合わせて処理
for key in jpg_files.keys():
    if key in csv_files:
        jpg_filename = jpg_files[key]
        csv_filename = csv_files[key]

        tif_filename = f"{key}_geo.tif"

        map_image=np.array(Image.open(os.path.join(folder_path,jpg_filename)))
        coords = pd.read_csv(os.path.join(folder_path, csv_filename),encoding = 'shift-jis',header=None)
        coords = coords.rename(columns={0:'col',1:'row',2:'lat',3:'lon'})

        coords['lat_decimal']=coords['lat'].map(convert_to_decimal)
        coords['lon_decimal']=coords['lon'].map(convert_to_decimal)
        ground_control_points = [GroundControlPoint(float(coords.loc[i]['row']),float(coords.loc[i]['col']),coords.loc[i]['lon_decimal'],coords.loc[i]['lat_decimal']) for i in range(len(coords))]

        out_path=os.path.join(folder_path,tif_filename)
        with rasterio.open(out_path,
                           'w',
                           gcps=ground_control_points,
                           height=map_image.shape[0],
                           width=map_image.shape[1],
                           count=3,
                           dtype=map_image.dtype,
                           driver='GTiff',
                           crs='EPSG:4301') as dst:
                           for i in range(3):
                            dst.write(map_image[:,:,i],i+1)
    else:
        pass


/usr/local/lib/python3.11/dist-packages/PIL/Image.py:3402: DecompressionBombWarning: Image size (98818628 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/PIL/Image.py:3402: DecompressionBombWarning: Image size (97227921 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/PIL/Image.py:3402: DecompressionBombWarning: Image size (98378150 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


In [6]:
#gdal.Warpによる座標系の確定。一旦EPSG=4301に確定する
# 入力のTIFFファイルが格納されたフォルダ
input_folder = '/content/drive/MyDrive/NorthKorea/'

# 出力フォルダ
output_folder = '/content/drive/MyDrive/NorthKorea/'


# TIFFファイルのリストを取得
tif_files = glob.glob(os.path.join(input_folder, '*_geo.tif'))


for input_geotiff in tif_files:
      base_name = os.path.basename(input_geotiff)
      name, extension = os.path.splitext(base_name)

    # 新しいファイル名を生成
      output_geo = name + '_S.tif'
      output_geotiff = os.path.join(output_folder, output_geo)

#      print(output_geotiff)
      ds = gdal.Open(input_geotiff)
      gdal.Warp(output_geotiff, ds,srcSRS="EPSG:4301",dstSRS="EPSG:4301")

In [7]:
#jpg画像に対応するcoodファイルから緯度経度の値を読み出し、その値を使用して画像の地図範囲を切り取る
folder_path = '/content/drive/MyDrive/NorthKorea/'# フォルダのパスを指定
csv_pattern = r"cood(\w+)\.csv"
tif_pattern = r"(\w+)_geo_S\.tif$"

tif_files = {}
csv_files = {}

# tifファイルを探して辞書に追加
for filename in os.listdir(folder_path):
    tif_match = re.match(tif_pattern, filename)
    if tif_match:
        key = tif_match.group(1)
        tif_files[key] = filename
#        print(filename)

# csvファイルを探して辞書に追加
for filename in os.listdir(folder_path):
    csv_match = re.match(csv_pattern, filename)
    if csv_match:
        key = csv_match.group(1)
        csv_files[key] = filename

# 共通のキーを持つファイルを組み合わせて処理
for key in tif_files.keys():
    if key in csv_files:
        tif_filename = tif_files[key]
        csv_filename = csv_files[key]

#        map_image=np.array(Image.open(os.path.join(folder_path,tif_filename)))
        coords = pd.read_csv(os.path.join(folder_path, csv_filename),encoding = 'shift-jis',header=None)
        coords = coords.rename(columns={0:'col',1:'row',2:'lat',3:'lon'})

        coords['lat_decimal']=coords['lat'].map(convert_to_decimal)
        coords['lon_decimal']=coords['lon'].map(convert_to_decimal)
#        ground_control_points = [GroundControlPoint(float(coords.loc[i]['row']),float(coords.loc[i]['col']),coords.loc[i]['lon_decimal'],coords.loc[i]['lat_decimal']) for i in range(len(coords))]

        minlat = min(coords['lat_decimal'])
        maxlat = max(coords['lat_decimal'])
        minlon = min(coords['lon_decimal'])
        maxlon = max(coords['lon_decimal'])
#        print(minlat,maxlat,minlon,maxlon)

        input_geotiff =os.path.join(folder_path,tif_filename)
        base_name = os.path.basename(input_geotiff)
        name, extension = os.path.splitext(base_name)

    # 新しいファイル名を生成
        output_geo = name + '_C.tif'
        output_geotiff = os.path.join(folder_path, output_geo)
        ds = gdal.Open(input_geotiff)
        ds1 = gdal.Translate(output_geotiff,ds,projWin=[minlon, maxlat, maxlon, minlat])
        ds1 =None


In [8]:
#EPSGを4301から4326に変換する
# 入力のTIFFファイルが格納されたフォルダ
folder_path = '/content/drive/MyDrive/NorthKorea/'

# フォルダ内のファイル一覧を取得
tif_files = glob.glob(os.path.join(folder_path, '*_geo_S_C.tif'))

for input_geotiff in tif_files:
    base_name = os.path.basename(input_geotiff)
#    name, extension = os.path.splitext(base_name)

    # 新しいファイル名を生成
    output_geo = base_name.replace('_geo_S_C.tif', '_geo_S_C_R.tif')
    output_geotiff = os.path.join(folder_path, output_geo)
    print(output_geotiff)

    ds = gdal.Open(input_geotiff)
    gdal.Warp(output_geotiff, ds, srcSRS="EPSG:4301", dstSRS="EPSG:4326")

/content/drive/MyDrive/NorthKorea/NI_52_9_3_geo_S_C_R.tif
/content/drive/MyDrive/NorthKorea/NI_53_13_12_geo_S_C_R.tif
/content/drive/MyDrive/NorthKorea/NI_53_22_10_geo_S_C_R.tif
/content/drive/MyDrive/NorthKorea/NK_54_22_14_geo_S_C_R.tif
/content/drive/MyDrive/NorthKorea/NL_55_17_16_geo_S_C_R.tif
/content/drive/MyDrive/NorthKorea/NL_55_24_11_geo_S_C_R.tif
